# Exploratory Data Analysis - Analyst Forecast Accuracy

In [10]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
import plotly
import plotly.express as px
import plotly.graph_objs as go
ek.set_app_key("f47c330480d74c598b7e8ebc2539424e91764dd8")

https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

### Accuracy Variables  

**TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

**TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

--> this is a analyst forecast variable

**TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

--> forecast error between actual EPS and TR.EPSMean  


### DataFrames

**df_accuracy** - basic dataframe containing quaterly data for all S&P 500 companies on EPS Actual, EPS Mean (the analyst forcast) and EPS Surprise (the forecast error in %)

**df_accuracy_new** - df_accuracy without extremely high or low values in the EPS Surprise column, so without outliers

**df_averages** - dataframe containing mean values of EPS Actual, EPS Mean and EPS Suprise over the entire time grouped by Instrument, takes df_accuracy_new as basis, so no outliers

**df_accuracy_yearly** - dataframe grouping quaterly datapoints into yearly data points

### 1) Summary statistics for EPS Actual, EPS Mean (forecast), and EPS Surprise

### Exploratory Data Analysis of Analyst Forcast Accuracy

In [49]:
df = pd.read_csv("Dataframes/accuracy_analysis_dataframe.csv")
df_accuracy = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual", "Earnings Per Share - Mean", "Earnings Per Share - Actual Surprise",]]
df_accuracy = df_accuracy.dropna()
df_accuracy["Date"] = pd.to_datetime(df_accuracy["Date"])
df_accuracy

Instrument                      Date  Earnings Per Share - Actual  \
0        POOL.OQ 2022-10-20 07:00:00+00:00                         4.78   
1        POOL.OQ 2022-07-21 07:00:00+00:00                         7.63   
2        POOL.OQ 2022-04-21 07:00:00+00:00                         4.23   
3        POOL.OQ 2022-02-17 07:00:00+00:00                         2.63   
4        POOL.OQ 2021-10-21 07:00:00+00:00                         4.51   
...          ...                       ...                          ...   
20115      AVY.N 2014-01-31 08:30:00+00:00                         0.69   
20116      AVY.N 2013-10-25 08:30:00+00:00                         0.69   
20117      AVY.N 2013-07-23 08:30:00+00:00                         0.71   
20118      AVY.N 2013-04-24 08:30:00+00:00                         0.59   
20119      AVY.N 2013-01-30 07:35:00+00:00                         0.54   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  
0                        4.58750                                 4.196  
1                        7.51700                                 1.503  
2                        3.14867                                34.342  
3                        1.87500                                40.267  
4                        3.84833                                17.194  
...                          ...                                   ...  
20115                    0.68000                                 1.471  
20116                    0.63833                                 8.095  
20117                    0.70250                                 1.068  
20118                    0.57571                                 2.482  
20119                    0.48571                                11.178  

[19429 rows x 5 columns]

In [50]:
df_accuracy.describe()

Earnings Per Share - Actual  Earnings Per Share - Mean  \
count                 19429.000000               19429.000000   
mean                      1.358353                   1.265157   
std                       2.954119                   2.808047   
min                     -16.430000                 -15.985000   
25%                       0.497780                   0.457730   
50%                       0.910000                   0.859170   
75%                       1.580000                   1.484460   
max                     123.650000                 126.765710   

       Earnings Per Share - Actual Surprise  
count                          1.942900e+04  
mean                           1.964045e+02  
std                            2.369133e+04  
min                           -8.858503e+03  
25%                            5.060000e-01  
50%                            4.589000e+00  
75%                            1.233300e+01  
max                            3.297926e+06

Min and max values seem very high. Next step is to check for outliers and remove them for better results:  

**Removing Outliers:**

In [51]:
#identifying outliers and replacing them with NA
summary_stats = df_accuracy["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df_accuracy["Earnings Per Share - Actual Surprise"].loc[~((df_accuracy["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df_accuracy["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df_accuracy.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new

Instrument                      Date  Earnings Per Share - Actual  \
0        POOL.OQ 2022-10-20 07:00:00+00:00                         4.78   
1        POOL.OQ 2022-07-21 07:00:00+00:00                         7.63   
2        POOL.OQ 2022-04-21 07:00:00+00:00                         4.23   
3        POOL.OQ 2022-02-17 07:00:00+00:00                         2.63   
4        POOL.OQ 2021-10-21 07:00:00+00:00                         4.51   
...          ...                       ...                          ...   
20115      AVY.N 2014-01-31 08:30:00+00:00                         0.69   
20116      AVY.N 2013-10-25 08:30:00+00:00                         0.69   
20117      AVY.N 2013-07-23 08:30:00+00:00                         0.71   
20118      AVY.N 2013-04-24 08:30:00+00:00                         0.59   
20119      AVY.N 2013-01-30 07:35:00+00:00                         0.54   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  
0                        4.58750                                 4.196  
1                        7.51700                                 1.503  
2                        3.14867                                34.342  
3                        1.87500                                40.267  
4                        3.84833                                17.194  
...                          ...                                   ...  
20115                    0.68000                                 1.471  
20116                    0.63833                                 8.095  
20117                    0.70250                                 1.068  
20118                    0.57571                                 2.482  
20119                    0.48571                                11.178  

[19429 rows x 5 columns]

In [52]:
na_count = df_accuracy_new["Earnings Per Share - Actual Surprise"].isna().sum()
na_count

774

--> deteceted outliers at the given threshold

In [53]:
#removing rows with NA (outliers)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new

Instrument                      Date  Earnings Per Share - Actual  \
0        POOL.OQ 2022-10-20 07:00:00+00:00                         4.78   
1        POOL.OQ 2022-07-21 07:00:00+00:00                         7.63   
2        POOL.OQ 2022-04-21 07:00:00+00:00                         4.23   
3        POOL.OQ 2022-02-17 07:00:00+00:00                         2.63   
4        POOL.OQ 2021-10-21 07:00:00+00:00                         4.51   
...          ...                       ...                          ...   
20115      AVY.N 2014-01-31 08:30:00+00:00                         0.69   
20116      AVY.N 2013-10-25 08:30:00+00:00                         0.69   
20117      AVY.N 2013-07-23 08:30:00+00:00                         0.71   
20118      AVY.N 2013-04-24 08:30:00+00:00                         0.59   
20119      AVY.N 2013-01-30 07:35:00+00:00                         0.54   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  
0                        4.58750                                 4.196  
1                        7.51700                                 1.503  
2                        3.14867                                34.342  
3                        1.87500                                40.267  
4                        3.84833                                17.194  
...                          ...                                   ...  
20115                    0.68000                                 1.471  
20116                    0.63833                                 8.095  
20117                    0.70250                                 1.068  
20118                    0.57571                                 2.482  
20119                    0.48571                                11.178  

[18655 rows x 5 columns]

In [54]:
df_accuracy_new.describe()

Earnings Per Share - Actual  Earnings Per Share - Mean  \
count                 18655.000000               18655.000000   
mean                      1.399211                   1.309889   
std                       2.994569                   2.855433   
min                     -16.430000                 -15.985000   
25%                       0.530000                   0.497890   
50%                       0.940000                   0.890910   
75%                       1.610000                   1.519765   
max                     123.650000                 126.765710   

       Earnings Per Share - Actual Surprise  
count                          18655.000000  
mean                               7.010210  
std                               17.124123  
min                              -82.249000  
25%                                0.528500  
50%                                4.416000  
75%                               11.533000  
max                               95.084000

### 2) Forecast error distribution

**Surprise Distribution - with outliers**

(commented it out because it's not that relevant but in case we want to look at it)

In [95]:
'''
fig = px.histogram(df_accuracy, x="Earnings Per Share - Actual Surprise", nbins=1000, title="EPS Surprise (%) Distribution (with outliers)")
fig.update_layout(yaxis=dict(tickformat=".2%"))
fig.show()
'''

'\nfig = px.histogram(df_accuracy, x="Earnings Per Share - Actual Surprise", nbins=1000, title="EPS Surprise (%) Distribution (with outliers)")\nfig.update_layout(yaxis=dict(tickformat=".2%"))\nfig.show()\n'

**Surprise Distribution - without outliers**

In [74]:
fig = px.histogram(df_accuracy_new, x="Earnings Per Share - Actual Surprise", nbins=1000, title="EPS Surprise (%) Distribution (outliers removed)")
fig.update_layout(yaxis=dict(tickformat=".2%"))
fig.show()
#NEED TO CHECK WHY Y AXIS IS SO WIERD!

### 3) Average EPS Actual, EPS Mean and EPS Surprise per Instrument for our selected time period

In [59]:
df_averages = df_accuracy_new.groupby("Instrument").mean()
df_averages

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_33053/3471284302.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



Earnings Per Share - Actual  Earnings Per Share - Mean  \
Instrument                                                           
A.N                            0.752250                   0.703307   
AAL.OQ                         0.233590                   0.163107   
AAP.N                          1.967500                   1.932031   
AAPL.OQ                        0.765268                   0.706897   
ABBV.N                         1.832051                   1.780069   
...                                 ...                        ...   
YUM.N                          0.864750                   0.830887   
ZBH.N                          1.787179                   1.724524   
ZBRA.OQ                        2.330750                   2.193593   
ZION.OQ                        0.833077                   0.751918   
ZTS.N                          0.707250                   0.654174   

            Earnings Per Share - Actual Surprise  
Instrument                                        
A.N                                     7.177500  
AAL.OQ                                  6.568718  
AAP.N                                   2.184900  
AAPL.OQ                                 7.314100  
ABBV.N                                  3.334179  
...                                          ...  
YUM.N                                   5.464400  
ZBH.N                                   4.548769  
ZBRA.OQ                                 5.452150  
ZION.OQ                                 9.320923  
ZTS.N                                   7.327175  

[502 rows x 3 columns]

Summary statistics per instrument

In [57]:
df_averages.describe()

Earnings Per Share - Actual  Earnings Per Share - Mean  \
count                   502.000000                 502.000000   
mean                      1.357478                   1.270546   
std                       2.319588                   2.205547   
min                      -0.337297                  -0.343831   
25%                       0.597601                   0.565188   
50%                       1.005837                   0.936834   
75%                       1.562812                   1.438929   
max                      44.810500                  43.054369   

       Earnings Per Share - Actual Surprise  
count                            502.000000  
mean                               7.092907  
std                                6.423570  
min                              -54.155000  
25%                                3.927906  
50%                                6.044750  
75%                                9.742726  
max                               50.320318

In [58]:
# calculate the mean of forecast errors for each company at each quarter
df_mean = df_accuracy_new.groupby(['Instrument', pd.Grouper(key='Date', freq='Q')])['Earnings Per Share - Actual Surprise'].mean().reset_index()

# create a line chart for each company
fig = go.Figure()

for company in df_mean['Instrument'].unique():
    # filter the data for each company
    df_company = df_mean[df_mean['Instrument'] == company]
    
    # add the line chart for the company
    fig.add_trace(go.Scatter(x=df_company['Date'], y=df_company['Earnings Per Share - Actual Surprise'],
                             mode='lines', name=company))

# set the chart title and axis labels
fig.update_layout(title='Quarterly Forecast Error by Company',
                   xaxis_title='Date', yaxis_title='Earnings Per Share - Actual Surprise')

# show the chart
fig.show()

Quaterly Forecast Error by Company - Top 15 Companies in terms of Actual EPS

In [81]:
# calculate the mean of forecast errors for each company at each quarter
df_mean = df_accuracy_new.groupby(['Instrument', pd.Grouper(key='Date', freq='Q')])['Earnings Per Share - Actual Surprise'].mean().reset_index()

# sort by Earnings Per Share - Actual Surprise in descending order and select top 15 companies
top_companies = df_mean.sort_values('Earnings Per Share - Actual Surprise', ascending=False).head(15)['Instrument'].unique()

# create a line chart for each of the top 15 companies
fig = go.Figure()

for company in top_companies:
    # filter the data for each company
    df_company = df_mean[df_mean['Instrument'] == company]
    
    # add the line chart for the company
    fig.add_trace(go.Scatter(x=df_company['Date'], y=df_company['Earnings Per Share - Actual Surprise'],
                             mode='lines', name=company))

# set the chart title and axis labels
fig.update_layout(title='Quarterly Forecast Error by Company - Top 15',
                   xaxis_title='Date', yaxis_title='Earnings Per Share - Actual Surprise')

# show the chart
fig.show()


In [82]:
# calculate the mean of forecast errors for each company at each quarter
df_mean = df_accuracy_new.groupby(['Instrument', pd.Grouper(key='Date', freq='Q')])['Earnings Per Share - Actual Surprise'].mean().reset_index()

# sort by Earnings Per Share - Actual Surprise in ascending order and select worst 15 companies
worst_companies = df_mean.sort_values('Earnings Per Share - Actual Surprise', ascending=True).head(15)['Instrument'].unique()

# create a line chart for each of the worst 15 companies
fig = go.Figure()

for company in worst_companies:
    # filter the data for each company
    df_company = df_mean[df_mean['Instrument'] == company]
    
    # add the line chart for the company
    fig.add_trace(go.Scatter(x=df_company['Date'], y=df_company['Earnings Per Share - Actual Surprise'],
                             mode='lines', name=company))

# set the chart title and axis labels
fig.update_layout(title='Quarterly Forecast Error by Company - Worst 15',
                   xaxis_title='Date', yaxis_title='Earnings Per Share - Actual Surprise')

# show the chart
fig.show()


--> looking at the last two plots, it seems as if for top 15 companies the forecast error is a bit more often positive, and stronger positive, compared to worst companies and vice versa for the Worst 15 companies for which it is more often negative compared to the top companies

Quaterly Forecast Error by Industry:

In [ ]:
#Quaterly Forecast Error

### 4) Number of Instruments per surprise percentile range for each year

**df_accuracy_yearly** - df with outliers  
**df_accuracy_yearly_new** - df without outliers

In [62]:
# grouping data by year
accuracy_variables = ['TR.EPSactValue.date', 'TR.EPSActValue', "TR.EPSMean", "TR.EPSActSurprise"]
df_accuracy_yearly, e = ek.get_data('0#.SPX', accuracy_variables, parameters = {'SDate':'0','EDate':'-10','Period':'FY0','Frq':'FY'})
df_accuracy_yearly["Date"] = pd.to_datetime(df_accuracy_yearly["Date"]).dt.year
df_accuracy_yearly = df_accuracy_yearly.dropna()
#df_accuracy_yearly

In [89]:
# removing outliers from yearly df
df_accuracy_yearly_new = df_accuracy_yearly.copy()
df_accuracy_yearly_new["Date"] = pd.to_datetime(df_accuracy_yearly_new["Date"]).dt.year
summary_stats_yearly = df_accuracy_yearly_new["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats_yearly.loc['25%']
Q3 = summary_stats_yearly.loc['75%']
IQR = Q3 - Q1
threshold2 = 7 #1.5 standard
surprise_outliers_removed2 = df_accuracy_yearly_new["Earnings Per Share - Actual Surprise"].loc[~((df_accuracy_yearly_new["Earnings Per Share - Actual Surprise"] < (Q1 - threshold2 * IQR)) | (df_accuracy_yearly_new["Earnings Per Share - Actual Surprise"] > (Q3 + threshold2 * IQR)))]
df_accuracy_yearly_new = df_accuracy.copy()
df_accuracy_yearly_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed2
df_accuracy_yearly_new = df_accuracy_yearly_new.dropna()
df_accuracy_yearly_new["Date"] = pd.to_datetime(df_accuracy_yearly_new["Date"]).dt.year
#df_accuracy_yearly_new

**with** outliers - number of Instruments per surprise percentile group for each year

In [64]:
bins = [-10000,-100, -50, -20, -10, -5, 0, 5, 10, 20, 30, 40, 50, 60, 80, 100, 10000]
# Group the data by year and calculate the percentile counts for each year
df_percentiles = pd.DataFrame(index=range(df_accuracy_yearly["Date"].min(), df_accuracy_yearly["Date"].max()+1),
                              columns=[f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)])
for year in df_percentiles.index:
    df_year = df_accuracy_yearly[df_accuracy_yearly["Date"] == year]
    percentile_counts = pd.cut(df_year["Earnings Per Share - Actual Surprise"], bins=bins, labels=df_percentiles.columns).value_counts().sort_index()
    df_percentiles.loc[year] = percentile_counts.values
df_percentiles

-10000--100 -100--50 -50--20 -20--10 -10--5 -5-0  0-5 5-10 10-20 20-30  \
2012           1        0       0       2      2   27   78    3     3     0   
2013           3        1       4       5      8  102  268   35    12     7   
2014           1        1       4       7      6  125  278   23    10     5   
2015           1        3       5       5     10  126  274   26    11     8   
2016           3        2       8       2     17   99  281   32    22     4   
2017           3        4       8       6     11  101  293   34    13     2   
2018           2        0       4      11      8   97  309   33    12     8   
2019           3        1       6      10      7  122  298   26    14     1   
2020           2        2       6       4      9   98  300   35    22     6   
2021           2        5       9      10     12   72  258   65    40    11   
2022           2        0       7       8     13   98  316   30    11     4   
2023           2        1       6       7     18   89  207   22    10     3   

     30-40 40-50 50-60 60-80 80-100 100-10000  
2012     0     0     0     0      0         0  
2013     4     0     2     2      2         5  
2014     2     1     0     0      0         3  
2015     1     2     0     1      0         3  
2016     2     0     4     1      0         4  
2017     3     2     2     0      0         1  
2018     1     0     1     2      0         1  
2019     1     2     0     1      0         3  
2020     4     3     0     1      2         3  
2021     5     2     0     1      3         5  
2022     3     2     1     2      1         2  
2023     4     0     1     0      0         0

In [65]:
# Melt the DataFrame to long format
df_percentiles_plot = df_percentiles.reset_index().melt(id_vars="index", var_name="percentile", value_name="count")

# Create line plot
fig = px.line(df_percentiles_plot, x="index", y="count", color="percentile")

# Set plot title and axis labels
fig.update_layout(title="Development of EPS Surprise Percentiles",
                   xaxis_title="Year",
                   yaxis_title="Count")

# Show plot
fig.show()

**without** outliers - number of Instruments per surprise percentile group for each year

In [66]:
bins2 = [-50, -20, -10, -5, - 3, -2, -1, 0, 1, 2, 3, 5, 10, 20, 30, 40, 50]
# Group the data by year and calculate the percentile counts for each year
df_percentiles2 = pd.DataFrame(index=range(df_accuracy_yearly_new["Date"].min(), df_accuracy_yearly_new["Date"].max()+1),
                              columns=[f"{bins2[i]}-{bins2[i+1]}" for i in range(len(bins2)-1)])
for year in df_percentiles2.index:
    df_year2 = df_accuracy_yearly_new[df_accuracy_yearly_new["Date"] == year]
    percentile_counts2 = pd.cut(df_year2["Earnings Per Share - Actual Surprise"], bins=bins2, labels=df_percentiles2.columns).value_counts().sort_index()
    df_percentiles2.loc[year] = percentile_counts2.values

df_percentiles2

-50--20 -20--10 -10--5 -5--3 -3--2 -2--1 -1-0  0-1 1-2 2-3 3-5 5-10  \
2012       0       0      1     1     0     1    2    2   5   0   2    0   
2013       0       7      9    15    14    19   53  131  89  40  44   22   
2014       0       7      9    12    11    28   55  112  81  40  43   36   
2015       0       8     19     8     9    22   66  119  61  41  39   48   
2016       0      11      5    15     8    19   65  110  82  46  45   30   
2017       0       4     18    16    17    16   69  125  81  38  47   30   
2018       0       7     11    15    19    20   70  144  75  49  43   31   
2019       0       4     10    12    16    19   67  146  81  48  56   23   
2020       0       7      8     9    14    20   59  153  99  34  60   39   
2021       0       7     11    19    11    22   62  144  97  50  44   42   
2022       0       6     14    12     9    17   72  143  69  51  53   41   

     10-20 20-30 30-40 40-50  
2012     0     0     0     0  
2013    11     3     0     0  
2014    15     3     0     0  
2015    21     1     0     0  
2016    25     0     0     0  
2017    23     2     0     0  
2018    12     1     0     0  
2019    18     0     0     0  
2020    15     0     0     0  
2021    15     0     0     0  
2022    14     1     0     0

In [67]:
# Melt the DataFrame to long format
df_percentiles_plot2 = df_percentiles2.reset_index().melt(id_vars="index", var_name="percentile", value_name="count")

# Create line plot
fig = px.line(df_percentiles_plot2, x="index", y="count", color="percentile")

# Set plot title and axis labels
fig.update_layout(title="Development of EPS Surprise Percentiles",
                   xaxis_title="Year",
                   yaxis_title="Count")

# Show plot
fig.show()

--> no percentile group has particularly high variation over time. All groups stay within their amount "lane"

Heatmap for data without outliers

In [68]:
heatmap_trace = go.Heatmap(z=df_percentiles2.values,
                           x=df_percentiles2.columns,
                           y=df_percentiles2.index,
                           colorscale='Viridis')

# create the layout
layout = go.Layout(title='Earnings Surprise Percentiles',
                   xaxis_title='Percentile Range',
                   yaxis_title='Year')

# create the figure object and plot
fig = go.Figure(data=[heatmap_trace], layout=layout)
fig.show()

--> we can see that most forecast errors are close to 0 (in the range around 0) and that there is not much change over the years.

### 5) Yearly Average Forecast Error of S&P 500 Companies

In [69]:
df_accuracy_yearly

Instrument  Date  Earnings Per Share - Actual  Earnings Per Share - Mean  \
0       POOL.OQ  2023                         18.7                      18.78   
1       POOL.OQ  2022                        15.92                      15.19   
2       POOL.OQ  2021                         9.13                       8.45   
3       POOL.OQ  2020                          6.4                    6.34111   
4       POOL.OQ  2019                         5.62                       5.65   
...         ...   ...                          ...                        ...   
5528      AVY.N  2017                         4.02                      3.967   
5529      AVY.N  2016                         3.44                      3.369   
5530      AVY.N  2015                         3.11                    3.01875   
5531      AVY.N  2014                         2.68                      2.664   
5532      AVY.N  2013                         2.08                    2.01444   

      Earnings Per Share - Actual Surprise  
0                                   -0.426  
1                                    4.806  
2                                    8.047  
3                                    0.929  
4                                   -0.531  
...                                    ...  
5528                                 1.336  
5529                                 2.108  
5530                                 3.023  
5531                                 0.601  
5532                                 3.255  

[5333 rows x 5 columns]

**with** outliers - Yearly Average Forecast Error of S&P 500 Companies

In [83]:
# Group the data by year and calculate the mean error for each year
df_yearly_mean = df_accuracy_yearly.groupby("Date")["Earnings Per Share - Actual Surprise"].mean().reset_index()

# Create a line plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_yearly_mean ["Date"], y=df_yearly_mean ["Earnings Per Share - Actual Surprise"], mode="lines", name="Average S&P500 Forecast Error"))

# Set the title and axis labels
fig.update_layout(title="Yearly Average Forecast Error of S&P 500 Companies",
                   xaxis_title="Year",
                   yaxis_title="Forecast Error (%)")
fig.show()

**without** outliers - Yearly Average Forecast Error of S&P 500 Companies

In [90]:
# Group the data by year and calculate the mean error for each year
df_yearly_mean2 = df_accuracy_yearly_new.groupby("Date")["Earnings Per Share - Actual Surprise"].mean().reset_index()

# Create a line plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_yearly_mean2 ["Date"], y=df_yearly_mean2 ["Earnings Per Share - Actual Surprise"], mode="lines", name="Average S&P500 Forecast Error"))

# Set the title and axis labels
fig.update_layout(title="Yearly Average Forecast Error of S&P 500 Companies",
                   xaxis_title="Year",
                   yaxis_title="Forecast Error (%)")
fig.show()

--> pretty much very constant over the years, a dip in 2018. Maybe averaging the quaters for the year is not so indicative.

### 6) Quaterly Average Forecast Error of S&P 500 Companies

(without outliers)

In [92]:
# Group the data by quarter and calculate the mean error for each quarter
df_quaterly_mean = df_accuracy_new.groupby(pd.Grouper(key="Date", freq="Q"))["Earnings Per Share - Actual Surprise"].mean().reset_index()

# Create a line plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_quaterly_mean["Date"], y= df_quaterly_mean["Earnings Per Share - Actual Surprise"], mode='lines', name='Average Forecast Error'))

# Set the title and axis labels
fig.update_layout(title='Quarterly Average Forecast Error of S&P 500 Companies',
                   xaxis_title='Quarter',
                   yaxis_title='Forecast Error (%)')
fig.show()

--> looking at the quaterly average forecast error, it is usually at around +5% - +7% percent. 2020 - 2022 the error was much higher, probably due to the pandemic and geo-political uncertainty.